<a href="https://colab.research.google.com/github/Jadhav7850/jango/blob/master/Org_Safety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

import holidays

from pyspark.sql.types import *
from pyspark.sql.functions import lit, min, max, pandas_udf, PandasUDFType

import mlflow
import mlflow.sklearn
import shutil

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

import logging
logging.getLogger('py4j').setLevel(logging.ERROR)

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from pyspark.sql.functions import col
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_add, next_day
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import mlflow.sklearn



In [4]:
## 1load the historical dataset
#from pyspark.sql import SparkSession
#from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType



# Create a Spark session
spark = SparkSession.builder.getOrCreate()

# Define the schema for the historical dataset
history_schema = StructType([
    StructField('date', TimestampType()),
    StructField('store', IntegerType()),
    StructField('item', IntegerType()),
    StructField('sales', IntegerType())
])

# Load the historical dataset
history = spark.read.csv(
    '/content/reduced_dataset (2).csv',
    header=True,
    schema=history_schema
).cache()

# Create a temporary view for the historical dataset
history.createOrReplaceTempView('history_tmp')



In [5]:
# 2generate holiday information for years in the historical dataset and
# the following year should forecasts extend into that year


# Generate holiday information for years in the historical dataset and the following year
first_date, last_date = history.selectExpr("min(date)", "max(date)").first()

holidays_df = spark.createDataFrame(
    [(date, holiday) for date, holiday in holidays.UnitedStates(years=range(first_date.year, last_date.year + 2)).items()],
    ['date', 'holiday']
).orderBy('date')

holidays_pd = holidays_df.toPandas()
holidays_broadcast = holidays_pd




In [6]:
# Generate holiday information for years in the historical dataset and the following year
first_date, last_date = (
    history
        .selectExpr("min(date)", "max(date)")
        .first()
)

holidays_df = spark.createDataFrame(
    [(date, holiday) for date, holiday in holidays.UnitedStates(years=range(first_date.year, 2 + last_date.year)).items()],
    ['date', 'holiday']
).orderBy('date')

holidays_pd = holidays_df.toPandas()
holidays_broadcast = holidays_pd


In [7]:
print(holidays_broadcast)



          date                     holiday
0   2013-01-01              New Year's Day
1   2013-01-21  Martin Luther King Jr. Day
2   2013-02-18       Washington's Birthday
3   2013-05-27                Memorial Day
4   2013-07-04            Independence Day
..         ...                         ...
60  2018-10-08                Columbus Day
61  2018-11-11                Veterans Day
62  2018-11-12     Veterans Day (Observed)
63  2018-11-22                Thanksgiving
64  2018-12-25               Christmas Day

[65 rows x 2 columns]


In [8]:
#3

def get_forecast(keys, history_pd):
    # Read keys associated with grouped data
    store = keys[0]
    item = keys[1]
    days_to_forecast = keys[2]

    # Prepare grouped data for training
    history_pd = history_pd.dropna()
    history_pd.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

    # Acquire holiday dataset
    holidays_pd = holidays_df

    # Instantiate and configure Prophet model
    model = Prophet(
        interval_width=0.95,
        growth='linear',
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode='multiplicative',
        holidays=holidays_pd
    )

    # Train the model
    model.fit(history_pd[['ds', 'y']])

    # Save fitted model for later utilization (during evaluation)
    model_path = '/tmp/forecast/{0}_{1}'.format(store, item)
    shutil.rmtree(model_path, ignore_errors=True)
    mlflow.sklearn.save_model(model, model_path)

    # Make forecast dataset
    future_pd = model.make_future_dataframe(
        periods=days_to_forecast,
        freq='d',
        include_history=True
    )

    # Generate forecast
    forecast_pd = model.predict(future_pd)

    # Merge history and forecast datasets
    forecast_pd = forecast_pd.merge(history_pd, on='ds', how='left')

    # Assign store and item to results
    forecast_pd['store'] = store
    forecast_pd['item'] = item

    # Return results
    forecast_pd.rename(
        columns={
            'ds': 'date', 'y': 'sales',
            'yhat': 'sales_pred_mean', 'yhat_lower': 'sales_pred_lower', 'yhat_upper': 'sales_pred_upper'
        },
        inplace=True
    )

    return forecast_pd[['store', 'item', 'date', 'sales', 'sales_pred_mean', 'sales_pred_lower', 'sales_pred_upper']]




In [9]:
#Identical to the above code
def get_forecast(keys, history_pd):
    # Read keys associated with grouped data
    store = keys[0]
    item = keys[1]
    days_to_forecast = keys[2]

    # Prepare grouped data for training
    history_pd = history_pd.dropna()
    history_pd.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

    # Acquire holiday dataset
    holidays_pd = holidays_broadcast

    # Instantiate and configure Prophet model
    model = Prophet(
        interval_width=0.95,
        growth='linear',
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode='multiplicative',
        holidays=holidays_pd
    )

    # Train the model
    model.fit(history_pd[['ds', 'y']])

    # Save fitted model for later utilization (during evaluation)
    model_path = '/tmp/forecast/{0}_{1}'.format(store, item)
    shutil.rmtree(model_path, ignore_errors=True)
    mlflow.sklearn.save_model(model, model_path)

    # Make forecast dataset
    future_pd = model.make_future_dataframe(
        periods=days_to_forecast,
        freq='d',
        include_history=True
    )

    # Generate forecast
    forecast_pd = model.predict(future_pd)

    # Merge history and forecast datasets
    forecast_pd = forecast_pd.merge(history_pd, on='ds', how='left')

    # Assign store and item to results
    forecast_pd['store'] = store
    forecast_pd['item'] = item

    # Return results
    forecast_pd.rename(
        columns={
            'ds': 'date', 'y': 'sales',
            'yhat': 'sales_pred_mean', 'yhat_lower': 'sales_pred_lower', 'yhat_upper': 'sales_pred_upper'
        },
        inplace=True
    )

    return forecast_pd[['store', 'item', 'date', 'sales', 'sales_pred_mean', 'sales_pred_lower', 'sales_pred_upper']]

import mlflow.sklearn





In [10]:
#4Use of get_forecast function.(Code use)

# Define the historical sales data as a pandas DataFrame
history_data = {
    'date': ['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04'],
    'sales': [10, 15, 12, 8]
}
history_pd = pd.DataFrame(history_data)#Datetime format

# Convert the 'date' column to datetime type(format that represents date)
history_pd['date'] = pd.to_datetime(history_pd['date'])

# Define the keys variable(info)
keys = ['store1', 'item1', 7]

# Define the holiday dataset DataFrame
holidays_data = {
    'ds': [
        '2013-01-01', '2013-01-21', '2013-02-18', '2013-05-27', '2013-07-04',

    ],
    'holiday': [
        "New Year's Day", "Martin Luther King Jr. Day", "Washington's Birthday",
        "Memorial Day", "Independence Day",

    ]
}
holidays_pd = pd.DataFrame(holidays_data)

# Convert the 'ds' column to datetime type
holidays_pd['ds'] = pd.to_datetime(holidays_pd['ds'])#convert ds to datetime

# Assign the holiday dataset to holidays_broadcast
holidays_broadcast = holidays_pd

# Call the get_forecast function
#generates forecasts ,returns  DataFrame with forecasted sales data.
result = get_forecast(keys, history_pd)

# Merge history and forecast datasets
result = pd.concat([result, history_pd], axis=0, ignore_index=True)

# Print the result
#print(result)

INFO:prophet:n_changepoints greater than number of observations. Using 2.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7du4xoub/mgqnkycp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7du4xoub/vf_lsi95.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=80045', 'data', 'file=/tmp/tmp7du4xoub/mgqnkycp.json', 'init=/tmp/tmp7du4xoub/vf_lsi95.json', 'output', 'file=/tmp/tmp7du4xoub/prophet_modelpdkhno5e/prophet_model-20230710040138.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:01:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:01:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distut

In [11]:
forecast_pd = get_forecast(keys, history_pd)
print(forecast_pd)

INFO:prophet:n_changepoints greater than number of observations. Using 2.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7du4xoub/k9arqs59.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7du4xoub/f48kvepk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99167', 'data', 'file=/tmp/tmp7du4xoub/k9arqs59.json', 'init=/tmp/tmp7du4xoub/f48kvepk.json', 'output', 'file=/tmp/tmp7du4xoub/prophet_modelrc4ifrem/prophet_model-20230710040142.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:01:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:01:43 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


     store   item       date  sales  sales_pred_mean  sales_pred_lower  \
0   store1  item1 2013-01-01   10.0        10.000027         10.000027   
1   store1  item1 2013-01-02   15.0        15.000065         15.000065   
2   store1  item1 2013-01-03   12.0        12.000109         12.000109   
3   store1  item1 2013-01-04    8.0         8.000131          8.000131   
4   store1  item1 2013-01-05    NaN         0.838181          0.838181   
5   store1  item1 2013-01-06    NaN       -22.360321        -22.360324   
6   store1  item1 2013-01-07    NaN       -13.610753        -13.610755   
7   store1  item1 2013-01-08    NaN         9.921528          9.921526   
8   store1  item1 2013-01-09    NaN       -12.091487        -12.091489   
9   store1  item1 2013-01-10    NaN       -22.504478        -22.504482   
10  store1  item1 2013-01-11    NaN       -50.393172        -50.393182   

    sales_pred_upper  
0          10.000027  
1          15.000065  
2          12.000109  
3           8.00013

In [12]:
#5 Evaluate the forecast performance
#forecast_pd:-DataFrame containing the forecasted sales data.
def evaluate_forecast(keys, forecast_pd):
    # Read keys associated with grouped data
    forecast_date = keys[0]
    store = int(keys[1])
    item = int(keys[2])

    # Calculate MSE & RMSE
    mse = mean_squared_error(forecast_pd['sales'], forecast_pd['sales_pred_mean'])
    rmse = sqrt(mse)

    # Calculate MAE
    mae = mean_absolute_error(forecast_pd['sales'], forecast_pd['sales_pred_mean'])

    # Calculate MAPE
    mape = np.mean(np.abs((forecast_pd['sales'] - forecast_pd['sales_pred_mean']) / forecast_pd['sales'])) * 100

    # Assemble result set
    results = {
        'forecast_date': [forecast_date],
        'store': [store],
        'item': [item],
        'mse': [mse],
        'rmse': [rmse],
        'mae': [mae],
        'mape': [mape]
    }

    print(results)  # Print the results
    return pd.DataFrame(data=results)
    #evaluation_result = pd.DataFrame(data=results)
    #print(evaluation_result)  # Print the results
    #return evaluation_result


    #The goal of this code is to evaluate
    #the forecast performance and provide a summary of the evaluation metrics for a given forecast.






In [13]:
keys = ['forecast_date', 1, 2]  # Replace 'store' with an actual store value and 'item' with an actual item value
forecast_data = {
    'sales': [10, 15, 12, 8],#Actual sales
    'sales_pred_mean': [11, 14, 10, 9]#Forecasted sales values
}
forecast_pd = pd.DataFrame(forecast_data)

# Call the evaluate_forecast function
evaluation_result = evaluate_forecast(keys, forecast_pd)

# Print the evaluation result
print(evaluation_result)


{'forecast_date': ['forecast_date'], 'store': [1], 'item': [2], 'mse': [1.75], 'rmse': [1.3228756555322954], 'mae': [1.25], 'mape': [11.458333333333334]}
   forecast_date  store  item   mse      rmse   mae       mape
0  forecast_date      1     2  1.75  1.322876  1.25  11.458333


In [14]:
forecastresult=evaluate_forecast(keys, forecast_pd)

{'forecast_date': ['forecast_date'], 'store': [1], 'item': [2], 'mse': [1.75], 'rmse': [1.3228756555322954], 'mae': [1.25], 'mape': [11.458333333333334]}


In [15]:
print(forecastresult)

   forecast_date  store  item   mse      rmse   mae       mape
0  forecast_date      1     2  1.75  1.322876  1.25  11.458333


In [16]:
# 6define function to generate forecast cross-validation evaluation metrics
def evaluate_forecast_cv(keys, forecast_pd):
    # read keys associated with grouped data (Extracts)
    forecast_date = keys[0]
    store = int(keys[1])
    item = int(keys[2])
    days_to_forecast = int(keys[3])

    # retrieve trained model
    model_path = '/dbfs/tmp/forecast/{0}_{1}'.format(store, item)
    model = mlflow.sklearn.load_model(model_path)

    # calculate cv performance metrics
    crossval_pd = cross_validation(
        model,
        initial='1095 days',
        horizon='{0} days'.format(days_to_forecast)
    )
    perf_pd = performance_metrics(
        crossval_pd,
        metrics=['mse', 'rmse', 'mae', 'mape', 'mdape', 'coverage']
    )

    # prepare results
    result = perf_pd.copy()
    result['forecast_date'] = forecast_date
    result['store'] = store
    result['item'] = item
    result['horizon'] = result['horizon'].apply(lambda h: h.days)

    # return metrics
    return result[['forecast_date', 'store', 'item', 'horizon', 'mse', 'rmse', 'mae', 'mape', 'mdape', 'coverage']]



In [17]:
#evaluate_forecast_cv(keys, forecast_pd)

In [18]:
#Modified version of evaluate_forecast_cv
import mlflow.sklearn
import pandas as pd

def evaluate_forecast_cv(keys, forecast_pd):
    # Read keys associated with grouped data
    forecast_date = keys[0]
    store = keys[1]
    item = keys[2]

    # Calculate metrics
    mse = forecast_pd['mse'].values[0]
    rmse = forecast_pd['rmse'].values[0]
    mae = forecast_pd['mae'].values[0]
    mape = forecast_pd['mape'].values[0]

    # Assemble result set
    results = {
        'forecast_date': [forecast_date],
        'store': [store],
        'item': [item],
        'mse': [mse],
        'rmse': [rmse],
        'mae': [mae],
        'mape': [mape]
    }
    return pd.DataFrame(data=results)


In [22]:
#7
forecast_date = '2017-12-01'
spark.conf.set('spark.sql.shuffle.partitions', 500 )

# generate forecast for this data
forecasts = (
  history
  .where(history.date < forecast_date) # limit training data to prior to our forecast date
  .groupBy('store', 'item', lit(30).alias('days_to_forecast'))
    .applyInPandas(get_forecast, "store integer, item integer, date timestamp, sales float, sales_pred_mean float, sales_pred_lower float, sales_pred_upper float")
    .withColumn('forecast_date', lit(forecast_date).cast(TimestampType()))
    ).cache()

#calculates evaluation metrics for the generated forecasts
forecast_evals = (
  forecasts
    .select('forecast_date', 'store', 'item', 'sales', 'sales_pred_mean')
    .where(forecasts.date < forecasts.forecast_date)
    .groupBy('forecast_date', 'store', 'item')
    .applyInPandas(evaluate_forecast, "forecast_date timestamp, store integer, item integer, mse float, rmse float, mae float, mape float")
    )

#calculates cross-validation evaluation metrics for the forecasts
forecast_evals_cv = (
  forecasts
    .select('forecast_date', 'store', 'item', 'sales', 'sales_pred_mean')
    .where(forecasts.date < forecasts.forecast_date)
    .groupBy('forecast_date', 'store', 'item', lit(30).alias('days_to_forecast'))
    .applyInPandas(evaluate_forecast_cv, "forecast_date timestamp, store integer, item integer, horizon integer, mse float, rmse float, mae float, mape float, mdape float, coverage float")
    )

forecasts.createOrReplaceTempView('forecasts_tmp')
forecast_evals.createOrReplaceTempView('forecast_evals_tmp')
forecast_evals_cv.createOrReplaceTempView('forecast_evals_cv_tmp')


In [23]:
print(forecasts)
print(forecast_evals)
print(forecast_evals_cv)

DataFrame[store: int, item: int, date: timestamp, sales: float, sales_pred_mean: float, sales_pred_lower: float, sales_pred_upper: float, forecast_date: timestamp]
DataFrame[forecast_date: timestamp, store: int, item: int, mse: float, rmse: float, mae: float, mape: float]
DataFrame[forecast_date: timestamp, store: int, item: int, horizon: int, mse: float, rmse: float, mae: float, mape: float, mdape: float, coverage: float]


In [24]:
#8Save the forecast data as a CSV file using Spark.
# Create a Spark session
spark = SparkSession.builder \
    .appName("Forecast Persistence") \
    .getOrCreate()

# Set the default database
spark.sql("CREATE DATABASE IF NOT EXISTS solacc_safety_stock")
spark.sql("USE solacc_safety_stock")

# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS forecasts")

# Create the forecasts table
forecasts_df = spark.sql("""
    SELECT
        a.forecast_date,
        a.store,
        a.item,
        a.date,
        b.sales,
        a.sales_pred_mean,
        a.sales_pred_lower,
        a.sales_pred_upper
    FROM forecasts_tmp a
    INNER JOIN history_tmp b
        ON a.store=b.store AND a.item=b.item AND a.date=b.date
""")

# Save the forecasts table as a CSV file
forecasts_df.write.mode("overwrite").csv("/content/new_path_to_save_forecast_evals.csv", header=True)

# Stop the Spark session
 #spark.stop()





In [25]:
#9Code extension to read Spark DataFrame, convert to Pandas DataFrame, and save as a CSV file
# Create a Spark session
spark = SparkSession.builder \
    .appName("Forecast Persistence") \
    .getOrCreate()

# Set the default database
spark.sql("CREATE DATABASE IF NOT EXISTS solacc_safety_stock")
spark.sql("USE solacc_safety_stock")

# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS forecast_evals")

# Read the forecast_evals_tmp data from a local CSV file
forecast_evals_tmp_df = spark.read.csv("/content/new_path_to_save_forecast_evals.csv", header=True, inferSchema=True)

# Create the forecast_evals table
forecast_evals_tmp_df.createOrReplaceTempView("forecast_evals_tmp")
forecast_evals_df = spark.sql("SELECT * FROM forecast_evals_tmp")

# Convert Spark DataFrame to Pandas DataFrame
forecast_evals_pandas_df = forecast_evals_df.toPandas()

# Save the forecast_evals Pandas DataFrame as a CSV file locally
forecast_evals_pandas_df.to_csv("path_to_save_forecast_evals.csv", index=False)





/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [26]:
#10forecast_evals_cv_tmp_df
# Set the default database
spark.sql("CREATE DATABASE IF NOT EXISTS solacc_safety_stock")
spark.sql("USE solacc_safety_stock")

# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS forecast_evals_cv")

# Read the forecast_evals_cv_tmp data from a local CSV file
forecast_evals_cv_tmp_df = spark.read.csv("/content/new_path_to_save_forecast_evals.csv", header=True, inferSchema=True)

# Create the forecast_evals_cv table
forecast_evals_cv_tmp_df.createOrReplaceTempView("forecast_evals_cv_tmp")
forecast_evals_cv_df = spark.sql("SELECT * FROM forecast_evals_cv_tmp")

# Convert Spark DataFrame to Pandas DataFrame
forecast_evals_cv_pandas_df = forecast_evals_cv_df.toPandas()

# Save the forecast_evals_cv Pandas DataFrame as a CSV file locally
forecast_evals_cv_pandas_df.to_csv("path_to_save_forecast_evals_cv.csv", index=False)




/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [27]:
#11forecasts_df
# extract a subset of data from the forecasts_df
spark = SparkSession.builder.getOrCreate()

# Load data from a CSV file into a Spark DataFrame
forecasts_df = spark.read.csv('/content/new_path_to_save_forecast_evals.csv', header=True, inferSchema=True)

# Register the forecasts_df DataFrame as a temporary table
forecasts_df.createOrReplaceTempView("forecasts_table")

# Execute the SQL query to filter the data
query = """
SELECT
  date,
  sales,
  sales_pred_mean
FROM forecasts_table
WHERE
  store = 1 AND
  item = 1 AND
  YEAR(date) = 2017
"""

# Run the query and get the result as a DataFrame
result_df = spark.sql(query)

# Convert the result DataFrame to a Pandas DataFrame
pandas_df = result_df.toPandas()

# Display the result
print(pandas_df)








          date  sales  sales_pred_mean
0   2017-01-01     19        20.230690
1   2017-01-02     15        11.159161
2   2017-01-03     10        14.131730
3   2017-01-04     16        14.736224
4   2017-01-05     14        15.449884
..         ...    ...              ...
359 2017-12-26     16        14.065387
360 2017-12-27     14        14.827838
361 2017-12-28     19        15.707889
362 2017-12-29     15        17.629807
363 2017-12-30     27        19.861452

[364 rows x 3 columns]


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [28]:
#12percent days below and above forecast by model
# Restart the SparkSession
spark = SparkSession.builder.getOrCreate()

# Assuming you have loaded the data into a Spark DataFrame called `forecasts_df`

# Register the forecasts_df DataFrame as a temporary table
forecasts_df.createOrReplaceTempView("forecasts")

# Execute the SQL query to calculate percent days below and above the forecast by model
result_df = spark.sql("""
    SELECT
        store,
        item,
        COUNT(*) as dates_total,
        SUM(CASE WHEN sales < sales_pred_mean THEN 1 ELSE 0 END) as dates_below_forecast,
        SUM(CASE WHEN sales > sales_pred_mean THEN 1 ELSE 0 END) as dates_above_forecast,
        FORMAT_NUMBER(SUM(CASE WHEN sales < sales_pred_mean THEN 1 ELSE 0 END) / COUNT(*), '##.#%') as dates_below_forecast_pct,
        FORMAT_NUMBER(SUM(CASE WHEN sales > sales_pred_mean THEN 1 ELSE 0 END) / COUNT(*), '##.#%') as dates_above_forecast_pct
    FROM forecasts
    GROUP BY store, item
    ORDER BY store, item
""")

# Convert the percentages to double type
result_df = result_df.withColumn("dates_below_forecast_pct", col("dates_below_forecast_pct").cast(DoubleType()))
result_df = result_df.withColumn("dates_above_forecast_pct", col("dates_above_forecast_pct").cast(DoubleType()))

# Show the result
result_df.show()


+-----+----+-----------+--------------------+--------------------+------------------------+------------------------+
|store|item|dates_total|dates_below_forecast|dates_above_forecast|dates_below_forecast_pct|dates_above_forecast_pct|
+-----+----+-----------+--------------------+--------------------+------------------------+------------------------+
|    1|   1|       1825|                 937|                 888|                    null|                    null|
|    2|   1|       1825|                 955|                 870|                    null|                    null|
|    3|   1|       1825|                 944|                 881|                    null|                    null|
|    4|   1|       1825|                 939|                 886|                    null|                    null|
|    5|   1|       1825|                 961|                 864|                    null|                    null|
|    6|   1|        870|                 447|                 42

In [29]:
#13Calculates the cycle associated with each date in the "forecasts_df" DataFrame
# Restart the SparkSession
spark = SparkSession.builder.getOrCreate()

# Assuming you have loaded the data into a Spark DataFrame called `forecasts_df`

# Calculate the cycle with which each date is associated
result_df = forecasts_df.select(
    col("store"),
    col("item"),
    col("date"),
    next_day(date_add(col("date"), -7), "Thursday").alias("cycle_start"),
    next_day(date_add(col("date"), -1), "Wednesday").alias("cycle_end"),
    date_add(next_day(date_add(col("date"), -7), "Thursday"), -4).alias("order_date")
).orderBy("store", "item", "date")

# Show the result
result_df.show()


+-----+----+-------------------+-----------+----------+----------+
|store|item|               date|cycle_start| cycle_end|order_date|
+-----+----+-------------------+-----------+----------+----------+
|    1|   1|2013-01-01 00:00:00| 2012-12-27|2013-01-02|2012-12-23|
|    1|   1|2013-01-02 00:00:00| 2012-12-27|2013-01-02|2012-12-23|
|    1|   1|2013-01-03 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-04 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-05 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-06 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-07 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-08 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-09 00:00:00| 2013-01-03|2013-01-09|2012-12-30|
|    1|   1|2013-01-10 00:00:00| 2013-01-10|2013-01-16|2013-01-06|
|    1|   1|2013-01-11 00:00:00| 2013-01-10|2013-01-16|2013-01-06|
|    1|   1|2013-01-12 00:00:00| 2013-01-10|2013-01-16|2013-01

In [30]:
#14Calculates the "cycle_stock" for each combination of store, item, cycle_start, and cycle


# Restart the SparkSession
spark = SparkSession.builder.getOrCreate()

# Assuming you have loaded the data into a Spark DataFrame called `forecasts_df`

# Calculate cycle_stock
cycles_df = forecasts_df.select(
    col("store"),
    col("item"),
    next_day(date_add(col("date"), -7), "Thursday").alias("cycle_start"),
    next_day(date_add(col("date"), -1), "Wednesday").alias("cycle_end"),
    col("sales_pred_mean")
).distinct().where((F.year(col("cycle_start")) >= 2013) & (F.year(col("cycle_end")) <= 2017))

window_spec = Window.partitionBy("store", "item", "cycle_start", "cycle_end")

result_df = cycles_df.withColumn("cycle_stock", F.sum(col("sales_pred_mean")).over(window_spec)) \
    .orderBy("store", "item", "cycle_start")

# Show the result
result_df.show()


+-----+----+-----------+----------+---------------+-----------+
|store|item|cycle_start| cycle_end|sales_pred_mean|cycle_stock|
+-----+----+-----------+----------+---------------+-----------+
|    1|   1| 2013-01-03|2013-01-09|      10.236608| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|      7.7797976| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|      13.701619| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|      12.240011| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|      10.992496| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|      14.268439| 79.0632156|
|    1|   1| 2013-01-03|2013-01-09|       9.844245| 79.0632156|
|    1|   1| 2013-01-10|2013-01-16|      7.4127755|  76.753627|
|    1|   1| 2013-01-10|2013-01-16|      13.362855|  76.753627|
|    1|   1| 2013-01-10|2013-01-16|      10.716124|  76.753627|
|    1|   1| 2013-01-10|2013-01-16|      9.9207535|  76.753627|
|    1|   1| 2013-01-10|2013-01-16|       9.498183|  76.753627|
|    1|   1| 2013-01-10|2013-01-16|     

In [31]:
#15calculates various stock-related metrics for each cycle
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Assuming you have loaded the data into a Spark DataFrame called 'forecasts_df'

# Calculate cycle stock
cycles_df = forecasts_df.groupby('store', 'item', F.window('date', '7 days', '1 day')).agg(
    F.sum('sales_pred_mean').alias('cycle_stock'),
    F.sum('sales').alias('cycle_sales'),
    F.stddev('sales').alias('cycle_stddev')
)

# Filter cycles within the desired date range
cycles_df = cycles_df.filter((F.year(cycles_df['window']['start']) >= 2013) & (F.year(cycles_df['window']['end']) <= 2017))

# Calculate cycle days and z-score
cycles_df = cycles_df.withColumn('cycle_days', F.datediff(cycles_df['window']['end'], cycles_df['window']['start']) + 1)
cycles_df = cycles_df.withColumn('zscore', F.lit(1.6449))  # z-score for 95% SLE

# Calculate safety stock and required stock
cycles_df = cycles_df.withColumn('safety_stock', F.col('zscore') * F.sqrt('cycle_days') * F.col('cycle_stddev'))
cycles_df = cycles_df.withColumn('required_stock', F.col('cycle_stock') + F.col('safety_stock'))

# Sort the results
window_spec = Window.partitionBy('store', 'item').orderBy('window')
cycles_df = cycles_df.withColumn('cycle_start', F.col('window')['start'].cast('date'))
cycles_df = cycles_df.orderBy('store', 'item', 'cycle_start').drop('window')

# Display the result
cycles_df.show()





+-----+----+-----------------+-----------+------------------+----------+------+------------------+------------------+-----------+
|store|item|      cycle_stock|cycle_sales|      cycle_stddev|cycle_days|zscore|      safety_stock|    required_stock|cycle_start|
+-----+----+-----------------+-----------+------------------+----------+------+------------------+------------------+-----------+
|    1|   1|       79.9960246|         83|1.5735915849388862|         8|1.6449| 7.321103026964487| 87.31712762696449| 2013-01-01|
|    1|   1|       79.2866606|         79|1.7994708216848745|         8|1.6449| 8.372001608081204| 87.65866220808121| 2013-01-02|
|    1|   1|       79.0632156|         80|1.8126539343499315|         8|1.6449|  8.43333577315982| 87.49655137315983| 2013-01-03|
|    1|   1|       78.7868436|         75|1.6035674514745462|         8|1.6449| 7.460565139834534| 86.24740873983453| 2013-01-04|
|    1|   1|78.47211460000001|         71| 1.345185418269098|         8|1.6449| 6.25844795

In [32]:
#16 subquery p performs calculations on aggregated data.
result_df = spark.sql("""
SELECT
  p.store,
  p.item,
  p.cycle_start,
  p.cycle_stock,
  p.zscore * SQRT(p.cycle_days) * p.cycle_stddev as safety_stock,
  p.cycle_stock + (p.zscore * SQRT(p.cycle_days) * p.cycle_stddev) as required_stock,
  p.cycle_sales
FROM (
  SELECT
    a.store,
    a.item,
    a.cycle_start,
    DATEDIFF(a.cycle_end, a.cycle_start) + 1 as cycle_days,
    1.6449 as zscore, -- z-score for 95% SLE
    a.cycle_stock,
    a.cycle_stddev,
    a.cycle_sales
  FROM (
    SELECT
      x.store,
      x.item,
      y.cycle_start,
      y.cycle_end,
      SUM(x.sales_pred_mean) as cycle_stock,
      SUM(x.sales) as cycle_sales,
      AVG(x.sales) as cycle_stddev
    FROM forecasts x
    INNER JOIN (
      SELECT DISTINCT -- cycles
        store,
        item,
        date_add(next_day(date, 'THU'), -7) as cycle_start,
        date_add(next_day(date, 'WED'), -1) as cycle_end
      FROM forecasts
      ) y
      ON x.store=y.store AND x.item=y.item AND x.date BETWEEN y.cycle_start AND y.cycle_end
    WHERE year(y.cycle_start) >= 2016 AND year(y.cycle_end) <= 2017
    GROUP BY
      x.store,
      x.item,
      y.cycle_start,
      y.cycle_end
    ) a
  ) p
WHERE p.store = 1 AND p.item = 1
ORDER BY store, item, cycle_start
""")

result_df.show()




+-----+----+-----------+------------------+------------------+------------------+-----------+
|store|item|cycle_start|       cycle_stock|      safety_stock|    required_stock|cycle_sales|
+-----+----+-----------+------------------+------------------+------------------+-----------+
|    1|   1| 2016-01-07| 92.73539000000001| 57.75137471662471|150.48676471662472|         86|
|    1|   1| 2016-01-07|        196.526977| 78.01245316191019|274.53943016191016|        171|
|    1|   1| 2016-01-14|        194.509709| 88.04914304238987|282.55885204238984|        193|
|    1|   1| 2016-01-14| 90.08315199999998| 51.03609858678462| 141.1192505867846|         76|
|    1|   1| 2016-01-21| 90.93764200000001| 69.83887175033685|160.77651375033685|        104|
|    1|   1| 2016-01-21|        199.951837| 96.26098021732777|296.21281721732777|        211|
|    1|   1| 2016-01-28|          95.03491|62.452068007512764|157.48697800751276|         93|
|    1|   1| 2016-01-28|208.04831900000002| 88.0491430423898

In [33]:
# buffered forecast
# Register the "forecasts" table as a temporary view
forecasts.createOrReplaceTempView("forecasts")

# Execute the SQL query
result_df = spark.sql("""
SELECT
  q.store,
  q.item,
  SUM(CASE WHEN q.cycle_sales > q.cycle_stock THEN 1 ELSE 0 END) as cycles_above_forecast,
  SUM(CASE WHEN q.cycle_sales > q.required_stock THEN 1 ELSE 0 END) as cycles_above_buffered_forecast,
  ROUND(100.0 * SUM(CASE WHEN q.cycle_sales > q.cycle_stock THEN 1 ELSE 0 END) / COUNT(*), 1) as cycles_above_forecast_pct,
  ROUND(100.0 * SUM(CASE WHEN q.cycle_sales > q.required_stock THEN 1 ELSE 0 END) / COUNT(*), 1) as cycles_above_buffered_forecast_pct,
  ROUND(100.0 * (1.0 - SUM(CASE WHEN q.cycle_sales > q.required_stock THEN 1 ELSE 0 END) / COUNT(*)), 1) as service_level
FROM (
  SELECT
    p.store,
    p.item,
    p.cycle_start,
    p.cycle_stock,
    p.zscore * SQRT(p.cycle_days) * p.cycle_stddev as safety_stock,
    p.cycle_stock + (p.zscore * SQRT(p.cycle_days) * p.cycle_stddev) as required_stock,
    p.cycle_sales
  FROM (
    SELECT
      a.store,
      a.item,
      a.cycle_start,
      DATEDIFF(a.cycle_end, a.cycle_start)+1 as cycle_days,
      1.6449 as zscore, -- z-score for 95% SLE
      a.cycle_stock,
      a.cycle_stddev,
      a.cycle_sales
    FROM (
      SELECT
        x.store,
        x.item,
        y.cycle_start,
        y.cycle_end,
        SUM(x.sales_pred_mean) as cycle_stock,
        SUM(x.sales) as cycle_sales,
        STDDEV(x.sales) as cycle_stddev
      FROM forecasts x
      INNER JOIN (
        SELECT DISTINCT -- cycles
          store,
          item,
          next_day(date_add(date, -7), 'thursday') as cycle_start,
          next_day(date_add(date, -1), 'wednesday') as cycle_end
        FROM forecasts
      ) y
      ON x.store=y.store AND x.item=y.item AND x.date BETWEEN y.cycle_start AND y.cycle_end
      WHERE YEAR(y.cycle_start) >= 2013 AND YEAR(y.cycle_end) <= 2017
      GROUP BY
        x.store,
        x.item,
        y.cycle_start,
        y.cycle_end
    ) a
  ) p
) q
GROUP BY q.store, q.item
ORDER BY q.store, q.item
""")

# Show the result
result_df.show()




+-----+----+---------------------+------------------------------+-------------------------+----------------------------------+-------------+
|store|item|cycles_above_forecast|cycles_above_buffered_forecast|cycles_above_forecast_pct|cycles_above_buffered_forecast_pct|service_level|
+-----+----+---------------------+------------------------------+-------------------------+----------------------------------+-------------+
|    1|   1|                  134|                            15|                     51.5|                               5.8|         94.2|
|    2|   1|                  127|                            11|                     48.8|                               4.2|         95.8|
|    3|   1|                  122|                            14|                     46.9|                               5.4|         94.6|
|    4|   1|                  128|                             8|                     49.2|                               3.1|         96.9|
|    5|   1| 

# New Section